In [1]:
import pandas as pd
import requests
import math
import yfinance as yf
from termcolor import colored as cl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use("fivethirtyeight")

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

def calculate_atr(df, period):
    df['High-Low'] = df['High'] - df['Low']
    df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    df['ATR'] = df['TR'].rolling(period).mean()
    return df

def calculate_supertrend(df, period, multiplier):
    calculate_atr(df, period)
    hl2 = (df['High'] + df['Low']) / 2
    df['Final Upperband'] = hl2 + (multiplier * df['ATR'])
    df['Final Lowerband'] = hl2 - (multiplier * df['ATR'])
    df['Supertrend'] = np.nan
    
    for current in range(1, len(df)):
        previous = current - 1
        if df['Close'][current] > df['Final Upperband'][previous]:
            df['Supertrend'][current] = df['Final Lowerband'][current]
        elif df['Close'][current] < df['Final Lowerband'][previous]:
            df['Supertrend'][current] = df['Final Upperband'][current]
        else:
            df['Supertrend'][current] = df['Supertrend'][previous]
            if df['Supertrend'][current] == df['Final Upperband'][previous] and df['Close'][current] <= df['Final Upperband'][current]:
                df['Supertrend'][current] = df['Final Lowerband'][current]
            elif df['Supertrend'][current] == df['Final Lowerband'][previous] and df['Close'][current] >= df['Final Lowerband'][current]:
                df['Supertrend'][current] = df['Final Upperband'][current]
                
    df['Signal'] = 0
    df['Signal'][df['Close'] > df['Supertrend']] = 1
    df['Signal'][df['Close'] < df['Supertrend']] = -1
    df['Position'] = df['Signal'].shift(1)
    return df

def backtest_strategy(df):
    df['Market Returns'] = df['Close'].pct_change()
    df['Strategy Returns'] = df['Market Returns'] * df['Position']
    df['Cumulative Strategy Returns'] = (1 + df['Strategy Returns']).cumprod()
    return df['Cumulative Strategy Returns'].iloc[-1]

# Fetching Google's stock data
googl = yf.download('META', start='2015-01-01', end='2020-01-01')

best_profit = -np.inf
best_period = None
best_multiplier = None

# Grid search over periods and multipliers
for period in range(6, 30):  # Example: search periods from 6 to 14
    for multiplier in np.arange(2, 10, 0.5):  # Example: search multipliers from 2 to 4.5 in steps of 0.5
        googl_data = calculate_supertrend(googl.copy(), period, multiplier)
        googl_data = backtest_strategy(googl_data)
        final_profit = googl_data
        
        if final_profit > best_profit:
            best_profit = final_profit
            best_period = period
            best_multiplier = multiplier

print(f"Best Period: {best_period}, Best Multiplier: {best_multiplier}, Best Profit: {best_profit}")


[*********************100%%**********************]  1 of 1 completed


Best Period: 8, Best Multiplier: 2.0, Best Profit: 2.8992099030095964


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

def calculate_atr(df, period):
    df['High-Low'] = df['High'] - df['Low']
    df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    df['ATR'] = df['TR'].rolling(period).mean()
    return df

def calculate_supertrend(df, period, multiplier):
    calculate_atr(df, period)
    hl2 = (df['High'] + df['Low']) / 2
    df['Final Upperband'] = hl2 + (multiplier * df['ATR'])
    df['Final Lowerband'] = hl2 - (multiplier * df['ATR'])
    df['Supertrend'] = np.nan
    
    for current in range(1, len(df)):
        previous = current - 1
        if df['Close'][current] > df['Final Upperband'][previous]:
            df['Supertrend'][current] = df['Final Lowerband'][current]
        elif df['Close'][current] < df['Final Lowerband'][previous]:
            df['Supertrend'][current] = df['Final Upperband'][current]
        else:
            df['Supertrend'][current] = df['Supertrend'][previous]
            if df['Supertrend'][current] == df['Final Upperband'][previous] and df['Close'][current] <= df['Final Upperband'][current]:
                df['Supertrend'][current] = df['Final Lowerband'][current]
            elif df['Supertrend'][current] == df['Final Lowerband'][previous] and df['Close'][current] >= df['Final Lowerband'][current]:
                df['Supertrend'][current] = df['Final Upperband'][current]
    
    df['Signal'] = 0
    df['Signal'][df['Close'] > df['Supertrend']] = 1  # Buy
    df['Signal'][df['Close'] < df['Supertrend']] = -1  # Sell
    df['Position'] = df['Signal'].shift(1)
    return df

def backtest_strategy(df):
    df['Market Returns'] = df['Close'].pct_change()
    df['Strategy Returns'] = df['Market Returns'] * df['Position']
    df['Cumulative Market Returns'] = (1 + df['Market Returns']).cumprod()
    df['Cumulative Strategy Returns'] = (1 + df['Strategy Returns']).cumprod()
    return df

# Fetching Google's stock data
googl = yf.download('META', start='2015-01-01', end='2020-01-01')

# Calculating Supertrend and backtesting
googl_data = calculate_supertrend(googl,8,2)
googl_data = backtest_strategy(googl_data)

# Calculating final values for comparison
initial_investment = googl_data['Close'].iloc[0]
final_value_buy_and_hold = googl_data['Close'].iloc[-1]
final_value_strategy = initial_investment * googl_data['Cumulative Strategy Returns'].iloc[-1]

profit_buy_and_hold = final_value_buy_and_hold - initial_investment
profit_strategy = final_value_strategy - initial_investment

# Display results
print(f"Initial Investment: ${initial_investment:.2f}")
print(f"Final Buy and Hold Value: ${final_value_buy_and_hold:.2f}, Profit: ${profit_buy_and_hold:.2f}")
print(f"Final Strategy Value: ${final_value_strategy:.2f}, Profit: ${profit_strategy:.2f}")
print(f"Cumulative Market Returns: {googl_data['Cumulative Market Returns'].iloc[-1]*100:.2f}%")
print(f"Cumulative Strategy Returns: {googl_data['Cumulative Strategy Returns'].iloc[-1]*100:.2f}%")
